In [10]:
import face_recognition
import cv2
import os
import csv
import numpy as np
import pafy
from PIL import Image
import glob
import json

In [11]:
# Open the input movie file

url = 'https://www.youtube.com/watch?v=l3NjiaBgVas'
vPafy = pafy.new(url)
play = vPafy.getbest()
video = play.download()

input_movie = cv2.VideoCapture(video)
length = int(input_movie.get(cv2.CAP_PROP_FRAME_COUNT))

In [12]:
import pandas as pd
known_faces = pd.read_csv('file.csv', sep=',',header=None).values

In [13]:
map = pd.read_csv('dict_file.csv', sep=',',header=None).values

In [14]:
face_locations = []
face_encodings = []
frame_number = 0

current_path = os.getcwd()
counter = 0
face_names = []

In [15]:
print("Inizio processamento")
while True:
    
    # Grab a single frame of video
    input_movie.set(1, frame_number)
    ret, frame = input_movie.read()
    frame_number += 20

    # Quit when the input video file ends
    if not ret:
        print("Fine")
        break

    # Find all the faces and face encodings in the current frame of video
    face_locations = face_recognition.face_locations(frame)
    face_encodings = face_recognition.face_encodings(frame, face_locations)


    for face_encoding in face_encodings:
        # See if the face is a match for the known face(s)
        match = face_recognition.compare_faces(known_faces, face_encoding, tolerance=0.50)
        # name = None
        if True in match:
            index = match.index(True)
            name = map[index]
            face_names.append(name[1])
        # If you had more than 2 faces, you could make this logic a lot prettier
        # but I kept it simple for the demo

Inizio processamento
Fine


In [16]:
# All done!
input_movie.release()
cv2.destroyAllWindows()
# create_csv.CreateCsv(current_path + "/face_database/")
print (face_names)

['scarlett_johansson', 'scarlett_johansson', 'scarlett_johansson', 'chris_evans', 'chris_evans', 'chris_evans']


In [17]:
json.dumps(face_names)

'["scarlett_johansson", "scarlett_johansson", "scarlett_johansson", "chris_evans", "chris_evans", "chris_evans"]'